In [1]:
%load_ext autoreload
%autoreload 2

In [24]:
import pandas as pd
from IPython.display import Markdown

from src.test_ import FTest, TTest
from src.stats import LinearRegression, add_constant

In [25]:
columns = {
    'Сводные данные о продаже наличной иностранной '
    'валюты кредитными организациями физическим лицам (x1)': 'x1',
    'Курс рубля к доллару США (x2)': 'x2',
    'Цена нефти марки "Юралс" (x3)': 'x3',
    'Динамика потребительских цен по группам товаров и услуг '
    '(месяц к соответствующему месяцу предыдущего года,%)  (x4)': 'x4',
    'Объем денежных накоплений в РФ миллиард руб. (y)': 'y'
}

In [26]:
df = pd.read_excel('data/task.xlsx', sheet_name='Лист1', header=3, usecols=[1, 2, 3, 4, 5])
df = df.rename(columns=columns)

In [27]:
y = df['y']
multiple_params = ['x1', 'x2', 'x3', 'x4']
x = add_constant(df[multiple_params])

In [28]:
model = LinearRegression(y, x)

In [32]:
print(FTest(model).pvalue_test_report())

$ F_{набл} = 17.698 $<br>
$ p\text{-}value_F = 0.000 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.208 $
<br><br>
$ 0.000 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ отвергается - модель в целом значима.


In [35]:
Markdown(FTest(model).pvalue_test_report())

$ F_{набл} = 17.7 $<br>
$ p\text{-}value_F = 6.83e-06 $
<br><br>
$ \alpha = 0.05 $<br>
$ F_{табл_{0.05}} = 2.21 $
<br><br>
$ 6.83e-06 < 0.05 $<br>
$ p\text{-}value < \alpha \rightarrow $ гипотеза $ H_0 $ отвергается - модель в целом значима.